Устанавливаем все необходимое

In [ ]:
!apt-get update
!apt-get install hisat2
!hisat2 --version

!apt-get install sra-toolkit

!java -version
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip
!chmod a+x FastQC/fastqc
!./FastQC/fastqc --help

!pip install HTSeq
!htseq-count --version

In [ ]:
!time wget http://83.149.211.146:23194/~ivan/data/hse_teaching/rna_seq_2021.tgz

In [ ]:
!time tar -xzvf rna_seq_2021.tgz

In [ ]:
!wget https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz
!tar -xzvf mm10_genome.tar.gz

In [ ]:
!wget http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M25/gencode.vM25.annotation.gtf.gz
!gzip -d gencode.vM25.annotation.gtf.gz

## Основная часть

Сначала объявляю массив имен файлов, которые нужно анализировать

In [2]:
files = ["SRR3414629", "SRR3414630", "SRR3414631", "SRR3414635", "SRR3414636", "SRR3414637"]

Удобно совместить bash и Python, чтобы в цикле выполнить нужные операции

In [2]:
def process(file):
    !./FastQC/fastqc  ./rna_seq_2021/{file}.fastq
    !time hisat2 -p 3 -x mm10/genome -U ./rna_seq_2021/{file}.fastq -S {file}.sam  2>  {file}.hisat
    !cat {file}.hisat
    !grep -P '^@|NH:i:1$' {file}.sam > {file}.uniq.sam

    !rm -v {file}.sam
    !time htseq-count --format=sam --stranded=no {file}.uniq.sam  gencode.vM25.annotation.gtf > {file}.counts

In [ ]:
for f in files:
    process(f+"_1")

Вытащим статистику из файлов и вычислим кол-во попавших на гены картирований 

In [16]:
uc_arr = []
counts_arr = []

def get_stats(file):
    uc = !grep -v '^@' {file}.uniq.sam | wc -l
    counts = !grep '^__' {file}.counts

    uc_arr.append(uc)
    counts_arr.append(counts)

for f in files:
    get_stats(f + "_1")

Для этих значений составил промежуточную таблицу, более простые (общее кол-во уникальных/нет, проценты и т.д. перенес руками в отчет)

In [26]:
counts_arr_mod = np.array([[y.split('\t')[1] for y in x] for x in counts_arr])[:, :2]
final_array = np.append(uc_arr, counts_arr_mod, axis=1).astype(np.int32)
hit_array = final_array[:, :1] - final_array[:, 1:2] - final_array[:, 2:3]
final_array = np.append(final_array, hit_array, axis=1)
final_array = np.append(np.array(files)[np.newaxis].T, final_array, axis=1)
hit_df = pd.DataFrame(final_array, columns=["id", "unique", "no_feature", "ambiguous", "hit"])
hit_df


,id,unique,no_feature,ambiguous,hit
0,SRR3414629,18375888,1604107,722172,16049609
1,SRR3414630,13186139,1240295,480520,11465324
2,SRR3414631,20928945,1700354,819740,18408851
3,SRR3414635,18428317,1392186,760134,16275997
4,SRR3414636,17825380,1332692,735108,15757580
5,SRR3414637,17844858,1397650,710230,15736978


In [8]:
import pandas as pd

allcounts = pd.read_csv(files[0] + "_1" + ".counts", sep='\t', names=['id', f'{files[0]}'])

for f in files[1:]:
    add = pd.read_csv(f + "_1" + ".counts", sep='\t', names=['id', f'{f}'])
    allcounts = allcounts.merge(add, how="left", on="id")

allcounts.columns = ['id', 'c1', 'c2', 'c3', 'r1', 'r2', 'r3']
allcounts.set_index('id', inplace=True)
allcounts.index.name = "geneID"
allcounts.head()

,c1,c2,c3,r1,r2,r3
geneID,,,,,,
ENSMUSG00000000001.4,4489,3919,5700,3431,3504,4031
ENSMUSG00000000003.15,0,0,0,0,0,0
ENSMUSG00000000028.15,345,273,468,150,136,152
ENSMUSG00000000031.16,64504,33249,64992,55526,48225,56064
ENSMUSG00000000037.17,77,68,87,41,44,52


In [9]:
writer = open("ALL.counts", "w")
writer.write(allcounts.to_csv(sep='\t'))
writer.close()

In [10]:
import numpy as np

allinfo_arr = np.array([allcounts.columns, 
files, ['control' if i < 3 else 'reprogramming' for i in range(6)]])
allinfo_arr = np.transpose(allinfo_arr)
allinfo = pd.DataFrame(allinfo_arr, columns=['index','id','condition'])
allinfo.set_index('index', inplace=True)
allinfo.index.name = 'sample'
allinfo

,id,condition
sample,,
c1,SRR3414629,control
c2,SRR3414630,control
c3,SRR3414631,control
r1,SRR3414635,reprogramming
r2,SRR3414636,reprogramming
r3,SRR3414637,reprogramming


In [14]:
writer = open("ALL.info", "w")
writer.write(allinfo.to_csv(sep='\t'))
writer.close()